# Roman/Rubin Observing Season Complementarity

The near-IR Nancy Grace Roman Space Telescope and the optical Vera C. Rubin Observatory both have the capacity to deliver deep limiting magnitude, high spatial resolution, timeseries photometry at complementary wavelengths.  This notebook explores how coordinating the observing schedules of these facility could be scientifically beneficial and outlines a metric for evaluating and optimizing complementary scheduling.  

This notebook specifically explores the scheduling of the Roman Galactic Bulge Time Domain Survey, but the metric is designed such that the same principle could be applied to other survey regions as well.  

In [8]:
from os import path
from roman_mission import RomanGalBulgeSurvey
from astropy.time import Time, TimeDelta
from astropy.coordinates import SkyCoord
import astropy.units as u
import numpy as np
import rubin_visibility
import rubin_sim.maf as maf
import healpixel_functions

## Roman Galactic Bulge Time Domain Survey (RGBTDS)

Roman is expected to observe the Galactic Bulge in seasons of approximately 60-70d long.  Due to pointing constraints of this telescope, these seasons will occur between February to April and August to October.  A total of ~6 seasons are anticipated, although the exact scheduling of these seasons is still to be determined.  Assuming the facility is launched on schedule in 2025 the first such season could take place, post-launch testing phase, in 2026.  

For the purposes of exploration, nominal dates for the RGBTDS seasons are defined with the following function.  
(Note: this produces warnings because future leap seconds are undefined; this does not significantly impact these calculations).

In [3]:
roman_survey = RomanGalBulgeSurvey()
print('Dates of nominal RGBTDS seasons: ')
for season in roman_survey.seasons:
    print(season)
print('Dates of season gaps:')
for gap in roman_survey.season_gaps:
    print(gap)

Dates of nominal RGBTDS seasons: 
{'start': <Time object: scale='utc' format='isot' value=2026-02-12T00:00:00.000>, 'end': <Time object: scale='utc' format='isot' value=2026-04-24T00:00:00.000>}
{'start': <Time object: scale='utc' format='isot' value=2026-09-19T00:00:00.000>, 'end': <Time object: scale='utc' format='isot' value=2026-10-29T00:00:00.000>}
{'start': <Time object: scale='utc' format='isot' value=2027-02-12T00:00:00.000>, 'end': <Time object: scale='utc' format='isot' value=2027-04-24T00:00:00.000>}
{'start': <Time object: scale='utc' format='isot' value=2027-09-19T00:00:00.000>, 'end': <Time object: scale='utc' format='isot' value=2027-10-29T00:00:00.000>}
{'start': <Time object: scale='utc' format='isot' value=2028-02-12T00:00:00.000>, 'end': <Time object: scale='utc' format='isot' value=2028-04-24T00:00:00.000>}
{'start': <Time object: scale='utc' format='isot' value=2028-09-19T00:00:00.000>, 'end': <Time object: scale='utc' format='isot' value=2028-10-29T00:00:00.000>}


## Rubin Observatory

Rubin Observatory is expected to start science operations in mid-2025 [see https://www.lsst.org/about/project-status], which for the sake of these calculations we take to be July 2025.  It will then spend 10 yrs conducting the Legacy Survey of Space and Time (LSST), which will survey the whole Southern Sky in 6 Sloan filters.  The exact cadence with which different areas of the survey footprint will be surveyed is a work in progress, combining many different science goals.  Many simulations of Rubin operations ("OpSimS") have been conducted exploring alternative strategies, and the current nominal strategy is described by opsim baseline_3.0.  

For our purposes, the important aspect of this opsim is that it includes timeseries observations of the RGBTDS footprint.  

The timestamps of the observations realized in any given strategy can be extracted from the opsim database using [Rubin's Metric Analysis Framework](https://www.lsst.org/scientists/simulations/maf) (MAF).  

## Inter-season Observations Metric

There will typically be gaps between sequential seasons of Roman observations of the Bulge.  These gaps will be at least several months long and potentially may exceed 1yr.  This means that the lightcurves of transient phenomena, including microlensing, will be partially sampled, in cases where the peak of the event occurs close to a season boundary.  This is particularly important for long-baseline events, such as those caused by black hole lenses, since regular sampling of the lightcurves is important to constrain parameters such as the microlensing parallax.  

The long baseline of LSST means that it could provide regular, if lower cadence, photometry of stars within the RGBTDS footprint during the inter-season gaps.  

The following metric is designed to run within the Rubin Observatory's MAF framework.  

In [63]:
class RomanRubinInterSeasonObsMetric(maf.metrics.BaseMetric):
    """Metric to evaluate whether LSST will provide observations during the 
    inter-season gaps in the Roman Galactic Bulge Time Domain Survey (RGBTDS).
    
    This metric calculates the median interval between sequential Rubin 
    observations in any filter, for all dates within the RGBTDS season gaps.  
    A signal-to-noise selection cut is applied to include only high limiting 
    magnitude observations.  
    
    The metric value returned represents the mean of this statistic over all season gaps. 
    """
    
    def __init__(self, 
                 time_col='observationStartMJD',
                 ra_col='fieldRA', 
                 dec_col='fieldDec', 
                 mag_cut=None,
                 nside=64,
                 metricName='RomanRubinInterSeasonObsMetric',
                 m5Col='fiveSigmaDepth',
                 **kwargs):
        
        # Store the column names of OpSim data columns for later use. 
        # Note a magnitude cut is applied to ensure that only high S/N observations are included.  
        # Although this is filter bandpass dependant, it simplifies the code considerably to consider 
        # a minimum value across all filters, with little loss of accuracy.
        self.ra_col = ra_col
        self.dec_col = dec_col
        self.mjdCol = time_col
        self.m5Col = m5Col
        self.mag_cut = 22.7
        self.nside = nside
        col_list = [self.ra_col, self.dec_col, self.mjdCol, self.m5Col]
        
        # Load information on the nominal RGBTDS observing seasons and field location
        self.roman_survey = RomanGalBulgeSurvey()
        
        super(RomanRubinInterSeasonObsMetric, self).__init__(col=col_list, metricName=metricName)
    
    def run(self, dataSlice, slicePoint):
        
        # Most metrics are designed to be run using a HEALpixSlicer, i.e. calculated for all points on the sky.
        # However, the RGBTDS footprint is entirely contained within a single HEALpixel with NSIDE=64.  
        # So we only proceed with this calculation if the current slicePoint contains the RGBTDS survey field. 
        # A given slicePoint contains the parameters of a particular slice, which for a HEALpixSlicer 
        # includes the specific HEALpixel ID, RA and Dec
        bulge_hp = healpixel_functions.skycoord_to_HPindex(self.roman_survey.bulge_field, self.nside)[0]
        if bulge_hp != slicePoint['sid']:
            metric = 0.0
        
        else:
            # For all sky pointings provided in the dataSlice, calculate the number of hours per night
            # for which Rubin can observe that sky pointing for dates between the start and end 
            # of the gaps between RGBTDS observing seasons
            dt = TimeDelta(1, format='jd', scale=None)
            metric = []
            for gap in self.roman_survey.season_gaps:
                
                # Get array of dates within the season gap, at intervals of 1 night
                deltagap = gap['end'] - gap['start']
                ndates = np.ceil((deltagap/dt).value)
                dates = gap['start'] + dt * np.arange(0, ndates, 1)

                # Fetch the timestamps of high S/N Rubin observations that occur within the Roman data gap
                # OpSim JDs are stored as MJDs, so we adjust the gap start/end dates accordingly 
                tobs_ordered = dataSlice[self.mjdCol]
                tobs_ordered.sort()
                idx1 = np.where(tobs_ordered >= (gap['start'].jd - 2400000.0))[0]
                idx2 = np.where(tobs_ordered <= (gap['end'].jd - 2400000.0))[0]
                idx3 = np.where(dataSlice[self.m5Col] >= self.mag_cut)[0]
                idx = set(idx1).intersection(set(idx2))
                gap_obs_index = list(idx.intersection(set(idx3)))
                tobs_ordered_gap = tobs_ordered[gap_obs_index]
                
                # If no observations occur within the gap, we set the metric value to the 
                # length of the gap
                if len(gap_obs_index) == 0:
                    metric.append(gap['end'].jd - gap['start'].jd)
                else:
                    
                # Calculate the median interval between sequential Rubin observations in any filter
                # in the Roman season gap:
                    delta_tobs = tobs_ordered_gap[1:] - tobs_ordered_gap[0:-1]
                    metric.append(np.median(delta_tobs))
            
            # Take mean of the metric over all season gaps:
            metric = np.median(np.array(metric))

        return metric
    

As an example, we can apply this metric to a simulation of Rubin's current baseline survey strategy, known as baseline_v3.0.  

In [64]:
opsim_db_file = '/Users/rstreet1/rubin_sim_data/sim_baseline/baseline_v3.0_10yrs.db'
runName = path.split(opsim_db_file)[-1].replace('.db', '')

# Load the OpSim database
opsim_db = maf.OpsimDatabase(opsim_db_file)

Run the metric calculations for all HEALpixels

In [65]:
bundleList = []
metric1 = RomanRubinInterSeasonObsMetric()
constraint = 'fiveSigmaDepth > 21.5'
slicer = maf.slicers.HealpixSlicer(nside=64, useCache=False)
plotDict = {'colorMax': 950}
bundleList.append(maf.MetricBundle(metric1, slicer, constraint, runName=runName, plotDict=plotDict))
bundleDict = maf.metricBundles.makeBundlesDictFromList(bundleList)
bundleGroup = maf.MetricBundleGroup(bundleDict, opsim_db, outDir='metric_results', resultsDb=None)
bundleGroup.runAll()

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Querying table None with constraint fiveSigmaDepth > 21.5 for columns ['rotSkyPos', 'fieldRA', 'fiveSigmaDepth', 'fieldDec', 'observationStartMJD']
Found 2038359 visits
Running:  ['baseline_v3_0_10yrs_RomanRubinInterSeasonObsMetric_fiveSigmaDepth_gt_21_5_HEAL']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


Now we can unpack the metric values.  Since a HEALpixSlicer is normally used with science metrics, the returned array will be NPIX long, but the value we are interested in is that for the HEALpix containing the RGBTDS survey field

In [66]:
metricRunName = 'baseline_v3_0_10yrs_RomanRubinInterSeasonObsMetric_fiveSigmaDepth_gt_21_5_HEAL'
metricMap = bundleDict[metricRunName].metricValues.data

bulge_hp = healpixel_functions.skycoord_to_HPindex(roman_survey.bulge_field, 64)[0]
metricValue = metricMap[bulge_hp]

print('Median interval between Rubin visits to the RGBTDS field during Roman season gaps = '+str(metricValue)+' days')

Median interval between Rubin visits to the RGBTDS field during Roman season gaps = 1.012864418058598 days
